<font size = 6>**RNN Model**</font>

# Import

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
from pyedflib import highlevel

# label

In [5]:
clic = pd.read_csv('../raw_data/clinical_information.csv')

In [6]:
clic.head()

,ID,EEG file,Gender,BW (g),GA (weeks),EEG to PMA (weeks),Diagnosis,Neuroimaging Findings,PNA of Imaging (days),Number of Reviewers Annotating Seizure,Primary Localisation,Other
0,1,eeg1,f,less than 2500g,37 to 38,37 to 38,mild/moderate asphyxia,widespread ischemic changes,0 to 4,3,both hemispheres; alternating,NaN
1,2,eeg2,m,less than 2500g,35 to 36,37 to 38,prematurity,ultrasound normal,0 to 4,1,NaN,NaN
2,3,eeg3,m,NaN,40 to 41,41 to 42,NaN,NaN,NaN,0,NaN,NaN
3,4,eeg4,m,3000 to 3500g,39 to 40,39 to 40,mild/moderate asphyxia,bilateral watershed area infarction,4 to 7,3,right centro-parietal,NaN
4,5,eeg5,f,2500 to 3000g,39 to 40,39 to 40,asphyxia (undefined grade); neonatal convulsions,haemorrhage in left lateral ventricle; subarac...,0 to 4,3,left hemisphere,NaN


In [7]:
tot_agree = clic[(clic["Number of Reviewers Annotating Seizure"]== 0) | (clic["Number of Reviewers Annotating Seizure"]== 3)]
tot_agree.head()

,ID,EEG file,Gender,BW (g),GA (weeks),EEG to PMA (weeks),Diagnosis,Neuroimaging Findings,PNA of Imaging (days),Number of Reviewers Annotating Seizure,Primary Localisation,Other
0,1,eeg1,f,less than 2500g,37 to 38,37 to 38,mild/moderate asphyxia,widespread ischemic changes,0 to 4,3,both hemispheres; alternating,NaN
2,3,eeg3,m,NaN,40 to 41,41 to 42,NaN,NaN,NaN,0,NaN,NaN
3,4,eeg4,m,3000 to 3500g,39 to 40,39 to 40,mild/moderate asphyxia,bilateral watershed area infarction,4 to 7,3,right centro-parietal,NaN
4,5,eeg5,f,2500 to 3000g,39 to 40,39 to 40,asphyxia (undefined grade); neonatal convulsions,haemorrhage in left lateral ventricle; subarac...,0 to 4,3,left hemisphere,NaN
6,7,eeg7,m,2500 to 3000g,38 to 39,38 to 39,infarction,right arteria cerebri media and anterior infar...,0 to 4,3,right hemisphere,NaN


In [6]:
tot_agree.iloc[i:i+1]["Number of Reviewers Annotating Seizure"]

NameError: name 'i' is not defined

In [8]:
healthy = []
epilepsy = []
target = []
for i in range(len(tot_agree)):
    if tot_agree.iloc[i:i+1]["ID"].iloc[0] in [4,29,50]:
        continue
    if tot_agree.iloc[i:i+1]["Number of Reviewers Annotating Seizure"].iloc[0]==0:
        healthy.append(tot_agree.iloc[i:i+1]["ID"].iloc[0])
        target.append(0)
    else:
        epilepsy.append(tot_agree.iloc[i:i+1]["ID"].iloc[0])
        target.append(1)

In [15]:
id_p = healthy+epilepsy

# Preprocessing

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyedflib import highlevel
from scipy.io import loadmat
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyClassifier
from scipy.fftpack import fft, fftfreq, rfft, rfftfreq
from imblearn.over_sampling import SMOTE
from scipy.signal import resample

In [10]:
from sklearn.base import TransformerMixin, BaseEstimator

class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

In [11]:
# Highpass filter
def highpass_filter(signals, sample_rate=256, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sample_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

# Powerline filter
def notch_filter(signals, sample_rate=256, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sample_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

# Combination of all filters and Scaler
def filter_signals(signals, scaler, sample_rate=256, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sample_rate)
    filter_notch = notch_filter(signals, sample_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

def concat_target(filtered_signals, target, names_ele):
    signals = filtered_signals.T
    signals_df = pd.DataFrame(signals,columns=names_ele)
    if (not 'Resp Effort' in signals_df.columns) and (not 'ECG EKG' in signals_df.columns):
        signals_df.rename(columns = {'ECG EKG-REF':'ECG EKG','Resp Effort-REF':'Resp Effort'}, inplace=True)
    signals_df.drop(columns=['Resp Effort', 'ECG EKG'], inplace=True)
    return signals_df

In [12]:
new_tar=[]
n=1
path_eeg = f'../raw_data/eeg{n}.edf'
signals, signal_headers, header = highlevel.read_edf(path_eeg)
names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])]
filtered_signals = filter_signals(signals, CustomTranformer())
signals_df = concat_target(filtered_signals, target, names_ele)
test_1 = np.array(signals_df)
slice_w = np.array([test_1[i:i+(60*256)] for i in range(0,len(test_1)-60*256, 30*256)])
tar_ap = [target[0]]*slice_w.shape[0]
new_tar+=tar_ap

In [13]:
new_tar

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [141]:
slice_w = np.array([test_1[i:i+(60*256)] for i in range(0,len(test_1)-60*256, 30*256)])

In [146]:
slice_w2=slice_w

In [150]:
slice_w.shape

(232, 15360, 19)

In [149]:
np.vstack([slice_w,slice_w2]).shape

(464, 15360, 19)

In [19]:
new_tar=[]
array_list =[]
for i,n in enumerate(id_p):
    print(n)
    path_eeg = f'../raw_data/eeg{n}.edf'
    signals, signal_headers, header = highlevel.read_edf(path_eeg)
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])]
    filtered_signals = filter_signals(signals, CustomTranformer())
    signals_df = concat_target(filtered_signals, target, names_ele)
    pat = np.array(signals_df)
    slice_w = np.array([pat[i:i+(60*256)] for i in range(0,len(pat)-60*256, 30*256)])
    tar_ap = [target[i]]*slice_w.shape[0]
    arr_reshaped = slice_w.reshape(slice_w.shape[0], -1)
    with open('df.txt', 'a') as file:
        np.savetxt(file, arr_reshaped) 

3
10
18
27
28
30
32
35
37
42
45
48
49
53
55
57
58
59
60
70
72
1
5
7
9
11
13
14
15
16
17
19
20
21
22
25
31
34
36
38
39
40
41
44
47
51
52
62
63
66
67
69
71
73
75
76
77
78
79


In [ ]:
loaded_arr = np.loadtxt("df.txt")

In [ ]:
loaded_arr